<a href="https://colab.research.google.com/github/Ravikrishnan05/PrediscanMedtech_project/blob/main/fastapi_for_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%
# RUN THIS CELL FIRST to forcefully uninstall the entire conflicting PyTorch ecosystem.
!pip uninstall -y torch torchvision torchaudio triton flash-attn unsloth bitsandbytes xformers fastai

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
Found existing installation: fastai 2.7.19
Uninstalling fastai-2.7.19:
  Successfully uninstalled fastai-2.7.19


In [2]:
# %%
# NOW, RUN THE OFFICIAL INSTALLATION.
# It will install torch, triton, bitsandbytes, and flash-attn correctly.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8ghrqt9r/unsloth_8c9357ba9f2845e192dd7a41cd7056c5
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8ghrqt9r/unsloth_8c9357ba9f2845e192dd7a41cd7056c5
  Resolved https://github.com/unslothai/unsloth.git to commit c1b73fa8836aa7e8b9ee13d748369f8f61e1fac5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.4 MB/s eta 0:

In [1]:
# %%
# After Unsloth is installed, we can install compatible versions of these libraries.
!pip install torchvision torchaudio

  Using cached torch-2.7.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached triton-3.3.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 67.3 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.3.0
    Uninstalling triton-3.3.0:
      Successfully uninstalled triton-3.3.0
  Attempting uninstall: torch
    Found existing installation: torch 2.7.0
    Uninstalling torch-2.7.0:
      Successfully uninstalled torch-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.6.1 re

In [2]:
# %%
# AFTER RESTARTING, RUN THIS CELL DIRECTLY

import torch
from unsloth import FastLanguageModel
from transformers import AutoProcessor

# Verification
import unsloth
print("✅ Unsloth and its dependencies are loaded correctly!")
print("Unsloth version:", unsloth.__version__)
print("PyTorch version:", torch.__version__)
if torch.cuda.is_available():
    # Let's also check torchvision, which we will install
    try:
        import torchvision
        print("Torchvision version:", torchvision.__version__)
    except ImportError:
        print("Installing compatible torchvision...")
        !pip install torchvision torchaudio
        import torchvision
        print("Torchvision version:", torchvision.__version__)


# --- Model Initialization ---
model_id = "google/medgemma-4b-it"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,
    dtype = dtype,
    load_in_4bit = True,
)
model.eval()

processor = AutoProcessor.from_pretrained(model_id)

print("\n✅ Model and processor loaded successfully!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
✅ Unsloth and its dependencies are loaded correctly!
Unsloth version: 2025.6.1
PyTorch version: 2.7.1+cu126
Torchvision version: 0.22.1+cu126
==((====))==  Unsloth 2025.6.1: Fast Gemma3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.12G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]


✅ Model and processor loaded successfully!


In [3]:
# %%
# Install the necessary packages for running the web API
!pip install fastapi uvicorn "python-multipart<0.0.7" pyngrok nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: python-multipart
    Found existing installation: python-multipart 0.0.20
    Uninstalling python-multipart-0.0.20:
      Successfully uninstalled python-multipart-0.0.20
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.31.0 requires python-multipart>=0.0.18, but you have python-multipart 0.0.6 which is incompatible.


In [4]:
# %%
# --- FINAL-FINAL-ABSOLUTELY-WORKS CELL ---

# Necessary imports
import uvicorn
import threading
import time
from pyngrok import ngrok
import nest_asyncio
import requests
import os

# --- Your FastAPI App Definition ---
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from PIL import Image
import torch
import io
import traceback

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True, allow_methods=["*"], allow_headers=["*"],
)

@app.get("/")
def read_root():
    return {"message": "✅ API is working!"}

@app.post("/generate")
async def generate(prompt: str = Form(...), image: UploadFile = File(...)):
    try:
        image_bytes = await image.read()
        pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        messages = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": prompt}]}]

        prompt_text = processor.apply_chat_template(messages, add_generation_prompt=True)
        inputs = processor(text=prompt_text, images=pil_image, return_tensors="pt").to(model.device)

        # =============================================================
        # THE NEW FIX: Simplest possible call to model.generate
        # This is the raw, underlying way to do it. It bypasses any
        # potential issues with Unsloth's patched `generate`.
        # =============================================================
        with torch.inference_mode():
            # We are now calling the original, un-patched forward method
            output = model.forward(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                pixel_values=inputs.get("pixel_values"),
            )
            # The logic for generating the next token is more manual but robust
            next_token_logits = output.logits[:, -1, :]
            next_token_id = torch.argmax(next_token_logits, dim=-1)

            # For a simple single-token response, this is enough.
            # For a full generation loop, it would be more complex, but let's
            # first prove we can get one token out without error.
            # To simplify, we'll just use the full generate function but with a `with` block.

            generated_ids = model.generate(
                **inputs,
                max_new_tokens=512,
                pad_token_id=tokenizer.pad_token_id,
            )

        # =============================================================

        generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
        response_text = generated_texts[0].split("Assistant: ")[-1].strip()
        return JSONResponse(content={"response": response_text})

    except Exception as e:
        print(traceback.format_exc())
        return JSONResponse(status_code=500, content={"error": str(e)})

# --- Server Launch Logic ---
def run_server():
    uvicorn.run(app, port=8000)

# IMPORTANT: Disconnect any existing ngrok tunnels before starting
ngrok.kill()
ngrok.set_auth_token("2yH8AJ7M9rJNbLfQoRANIuEudom_5Ppznh7REV7PvB51AaZXe")
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(f"🌐 Public API URL: {public_url}")

server_thread = threading.Thread(target=run_server)
server_thread.daemon = True
server_thread.start()

time.sleep(5)
print("✅ Server should be running!")

# --- Localhost Test Logic ---
print("\n" + "="*50)
print("🚀 LAUNCHING LOCALHOST TEST")
print("="*50)

local_api_url = "http://127.0.0.1:8000"
image_path = "/content/1.png"

if not os.path.exists(image_path):
    print(f"❌ Error: Test image '{image_path}' not found. Please upload it.")
else:
    endpoint = f"{local_api_url}/generate"
    data = {"prompt": "What are the key findings in this fundus image"}
    files = {"image": (image_path, open(image_path, "rb"), "image/png")}

    print("🚀 Sending request directly to localhost...")
    try:
        response = requests.post(endpoint, data=data, files=files, timeout=300)
        response.raise_for_status()
        print("\n✅✅✅ VICTORY! API RESPONSE: ✅✅✅")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"\n❌ An error occurred during the local request: {e}")

🌐 Public API URL: NgrokTunnel: "https://9a67-34-169-177-220.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [1828]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


✅ Server should be running!

🚀 LAUNCHING LOCALHOST TEST
🚀 Sending request directly to localhost...
Traceback (most recent call last):
  File "<ipython-input-4-7d512df87732>", line 49, in generate
    output = model.forward(
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/unsloth_zoo/temporary_patches/gemma.py", line 339, in forward
    outputs = self.model(
              ^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1762, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py", line 969, in wrapper
    output = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/li

In [ ]:
#The following are kill switches to stop the server running(As sometimes dead server might run(zombie servers)).

In [10]:
# %%
# Find the process using port 8000
#!lsof -i :8000

COMMAND PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 289 root  105u  IPv4  97248      0t0  TCP localhost:8000 (LISTEN)
python3 289 root  107u  IPv4 170871      0t0  TCP localhost:57018->localhost:8000 (CLOSE_WAIT)


In [ ]:
# %%
# Replace <PID> with the number from the command above
#!kill -9 289